<div align="center">
    <img src= "/assets/content/datax_logos/DataX_blue_wide_logo.png" align="center" width="100%">
</div>

### **HOMEWORK** 02 - **FLASK** WTF 

<br>

<div align="center" style="font-size:12px; font-family:FreeMono; font-weight: 100; font-stretch:ultra-condensed; line-height: 1.0; color:#2A2C2B">
    <img src="/assets/content/images/flask_logo.png" align="center" width="20%" padding="10"><br>
    <br>
</div>

<br>

<br>

In **m320_hw_1--flask_minimal_application** we introduced [**routing (decorators)**](https://flask.palletsprojects.com/en/1.1.x/api/#flask.Flask.route), the [**extends**](https://flask.palletsprojects.com/en/1.1.x/patterns/templateinheritance/) statement, and built a **Flask package**. This homework exercise introduces [**web forms**](https://flask.palletsprojects.com/en/1.1.x/patterns/wtforms/) which allow users to submit blog posts, and enables logging into the application. 

In order to successfully complete this assignment, make sure you've watched the myFlaskApp.py walkthrough video, as it discusses in a general way what you are being asked to do. 

<br>

**Author list:** [Elias Casto Hernandez](https://www.linkedin.com/in/ehcastroh/)

**References and Additional Resources**
<br>


> * [Flask by Example by Real Python](https://realpython.com/flask-by-example-part-1-project-setup/#project-setup)

> * [Flask Mega Tutorial by Miguel Grinberg](https://courses.miguelgrinberg.com/courses/flask-mega-tutorial/lectures/5203689)

> * [Flask Deployment Options](https://flask.palletsprojects.com/en/1.1.x/deploying/#deployment)

<hr style="border: 2px solid#003262;" />

#### HW 2: INTRODUCTION TO FLASK-WTF

<div align="center" style="font-size:12px; font-family:FreeMono; font-weight: 100; font-stretch:ultra-condensed; line-height: 1.0; color:#2A2C2B">
    <img src="/assets/content/images/flask-03.png" align="center" width="30%" padding="10"><br>
    <br>
</div>

<br>

**Recall:** [**Flask**](https://flask.palletsprojects.com/en/1.1.x/) does not perform [**form validation**](https://developer.mozilla.org/en-US/docs/Learn/Forms/Form_validation#What_is_form_validation). The following takes the **Flask package** built in **m320_hw_1--flask_minimal_application** and introduces the [**Flask-WTF**](https://flask-wtf.readthedocs.io/en/stable/) extension, which is a wrapper built around the [**WTForms**](https://wtforms.readthedocs.io/en/2.3.x/) package.

This assignment assumes that hw_1 is fully working as requested, in *local host*. We will be building the following directory hierarchy:

```bash
/myFlaskPackage
/venv
/flaskApp
/__init__.py
/routes.py
/config.py       # new file created in hw_2
/templates
    /base.html
    /index.html
    /login.html  # new file created in hw_2
/static
    /CSS
        /main.css
```

___

### **TO-DO**

0) Install Flask-WTF

https://flask-wtf.readthedocs.io/en/stable/install.html

```bash
# make sure you are in a virtual environment 
(venv) $ pip install flask-wtf
```


>**Note:** You will need a virtual environment in order to continue. If you need help initializing your virtual environment, see **m320_Setting-up-Flask** for instructions on how to create the environment.


<br>

1) create a **Secret Key Configuration Class** named ```config.py``` -- we do this to ensure [separations of concerns](https://en.wikipedia.org/wiki/Separation_of_concerns). Then add the following:

```python
import os

class Config(object):
    # generate  signature/tokens by joining using 'or'
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'never-gonna-guess'
```

<br>

2) Update the ```__init__.py``` file to include the following:

```python
from flask import Flask
from config import Config

app = Flask(__name__)
app.config.from_object(Config)

from app import routes
```

<br>

3) Create a new module ```forms.py``` for handling **form validation**, and include the following:

```python
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, BooleanField, SubmitField
from wtforms.validators import DataRequired

class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])   # validators =[DataRequired()] ensures that it's not empty
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Remember Me')
    submit = SubmitField('Sign In')
```

<br>

4) Create a template, ```login.html```, for rendering form validation data (i.e. include the following):

```html
# note that we are extending base.html
{% extends "base.html" %}

{% block content %}
    <h1>Sign In</h1>
    <form action="" method="post" novalidate>
        {{ form.hidden_tag() }}
        <p>
            {{ form.username.label }}<br>
            {{ form.username(size=28) }}
        </p>
        <p>
            {{ form.password.label }}<br>
            {{ form.password(size=32) }}
        </p>
        <p>{{ form.remember_me() }} {{ form.remember_me.label }}</p>
        <p>{{ form.submit() }}</p>
    </form>
{% endblock %}

```

<br>

5) Update ```routes.py``` to handle **form views**:

```python

from flask import render_template
from app import app
from app.forms import LoginForm

@app.route('/login')
def login():
    form = LoginForm()
    # note parameters and compare to those from hw_1
    return render_template('login.html', title='Sign In', form=form)
```

<br>

6) Update ```routes.py``` to **receive form data**:

```python
from flask import render_template, flash, redirect
from app import app
from app.forms import LoginForm

@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    
    # the following is new
    if form.validate_on_submit():
        flash('Login requested for {}, remember_me={}'.format(
            form.username.data, form.remember_me.data))
        return redirect('/index')
    return render_template('login.html', title='Sign In', form=form)
```

<br>

7) Update ```base.html``` to **receive and display login fields** as well include crude navigation bar

```html
<!DOCTYPE html>
<!-- THIS HTML SERVES AS THE BASE TEMPLATE FOR SITE -->
<html lang="en">
<head>
	<!-- set html default global design -->
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">

    <!-- enable CSS design and pass main.css as string -->
    <link rel="stylesheet" herf="{{ url_for('static', filename='CSS/main.css')}}">

    <!-- check that site title is included, else default title -->
    {% if title %}
    <title>{{ title }} - site</title>
    {% else %}
    <title>Welcome to DataX - Site</title>
    {% endif %}
</head>
<body>
    <!-- create crude navigation bar with link generation-->
    <div>
        Microblog:
        <a href="{{ url_for('index') }}">Home</a>
        <a href="{{ url_for('login') }}">Login</a>
    </div>
    <hr>
        <!-- return all flashed messages -->
        {% with messages = get_flashed_messages() %}
        {% if messages %}
        <ul>
            {% for message in messages %}
            <li>{{ message }}</li>
            {% endfor %}
        </ul>
        {% endif %}
        {% endwith %}
        {% block content %}{% endblock %}
</body>
</html>
```

<br>

8) Improve **field validation** by updating ```login.html```:

```html
{% extends "base.html" %}

{% block content %}
    <h1>Sign In</h1>
    
    # form validation sends error message if username/password is invalid
    <form action="" method="post" novalidate>
        {{ form.hidden_tag() }}
        <p>
            {{ form.username.label }}<br>
            {{ form.username(size=32) }}<br>
            {% for error in form.username.errors %}
            <span style="color: red;">[{{ error }}]</span>
            {% endfor %}
        </p>
        <p>
            {{ form.password.label }}<br>
            {{ form.password(size=32) }}<br>
            {% for error in form.password.errors %}
            <span style="color: red;">[{{ error }}]</span>
            {% endfor %}
        </p>
        <p>{{ form.remember_me() }} {{ form.remember_me.label }}</p>
        <p>{{ form.submit() }}</p>
    </form>
{% endblock %}

```

<br>

9) Update ```routes.py``` to handle url generation:

```python
from flask import render_template, flash, redirect, url_for
from app import app
from app.forms import LoginForm

@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    
    # the following is new
    if form.validate_on_submit():
        flash('Login requested for {}, remember_me={}'.format(
            form.username.data, form.remember_me.data))
        
        # hard-coded way commented out
        #return redirect('/index')
        
        #url generation
        return redirect(url_for('index'))
    return render_template('login.html', title='Sign In', form=form)
```

<br>

<hr style="border: 2px solid#003262;" />

### **DELIVERABLES**

Please submit the following: 

>(1) Use the **m320_Setting-up-Flask** materials to create a virtual environment for work to be done.

>(2) Take a screen capture of your localhost site once all of the above changes have been made.

>(3) Submit the entire directory containing the updated and new files used to create the above.

<hr style="border: 2px solid#003262;" />